# Translate text to other languages

You will take the text chunks and questions that you have created in the previous notebook and translate them to other languages using an open weights model.

Define the target language you want to translate to.

In [13]:
target_language = "arabic"

### Download the model

Download the model in `GGUF` format from [Hugging Face](https://huggingface.co/bartowski/aya-expanse-32b-GGUF/tree/main). This requires more resources, so make sure you have enough RAM and VRAM.

Download the following file:

- `aya-expanse-32b-Q4_K_M.gguf`

After that serve the model using [llama-server](https://github.com/ggml-org/llama.cpp/blob/master/tools/server/README.md):

```bash
llama-server \
  --model ~/.cache/llama.cpp/aya-expanse-32b-Q4_K_M.gguf \
  --n-gpu-layers 5 \
  --ctx-size 4096 \
  --batch-size 2048 \
  --ubatch-size 2048 \
  --port 36912
```

If you run out of memory, try to offload less layers to the GPU by reducing the value of `--n-gpu-layers`.

### Create the system prompt

Create a system prompt to teach the LLM how to perform the translation task.

In [14]:
system_prompt = f"""
## Task description

You are a world-class translator. Your task is to translate a given text into a specified target language. Don't do a literal word-for-word translation; instead, ensure that the translated text captures the original meaning, tone, and context.

### Important instructions:

- Don't add or remove any information from the original text.
- Maintain the original tone and style of the text.
- Return only the translated text without any additional commentary or explanations.

Here is the text, please translate it into {target_language}:
"""

### Load the data

In [15]:
import json

prefix = "glide_the_state_of_ai_in_operations_2025_report_chunks"
text_chunk_file_path = f"../data/chunks/{prefix}.json"
questions_file_path = f"../data/question_answer_pairs/{prefix}_qa_pairs.json"

with open(text_chunk_file_path, "r") as f:
    text_chunks = json.load(f)

with open(questions_file_path, "r") as f:
    questions = json.load(f)

print(f"Loaded {len(text_chunks)} text chunks.")
print(f"Loaded {len(questions)} questions.")

Loaded 14 text chunks.
Loaded 135 questions.


### Prepare the endpoint

In [16]:
import requests

# Don't forget to start the llama.cpp server!
LLAMA_SERVER_URL = "http://localhost:36912"


def translate_text(system_prompt: str, text: str, max_tokens: int = 4096) -> str | None:
    url = f"{LLAMA_SERVER_URL}/v1/chat/completions"
    headers = {"Content-Type": "application/json"}

    messages = [
        {
            "role": "system",
            "content": system_prompt,
        },
        {"role": "user", "content": text},
    ]

    data = {
        "messages": messages,
        "max_tokens": max_tokens,
    }

    try:
        response = requests.post(url=url, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        response_json = response.json()

        translated_text = response_json["choices"][0]["message"]["content"].strip()
        return translated_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

### Translate the text

Translate the text chunks.

In [ ]:
from tqdm import tqdm

for chunk in tqdm(text_chunks, total=len(text_chunks)):
    text = chunk["text_chunk"]
    response = translate_text(system_prompt, text)
    if response is None:
        raise Exception("Translation failed.")

    chunk["text_chunk"] = response

This cell displays the Arabic text with proper right-to-left alignment to make it more readable.

In [32]:
from IPython.display import display, HTML

text = text_chunks[1]["text_chunk"]
text_with_breaks = text.replace("\n", "<br>")
display(
    HTML(
        f'<p dir="rtl" style="text-align: right; font-family: Arial, sans-serif; line-height: 1.8;">{text_with_breaks}</p>'
    )
)

Now, translate the questions.

In [ ]:
from tqdm import tqdm

for question_object in tqdm(questions, total=len(questions)):
    question_text = question_object["question"]
    response = translate_text(system_prompt, question_text)
    if response is None:
        raise Exception("Translation failed.")

    question_object["question"] = response.text

### Save the translated data

In [ ]:
translated_text_chunk_file_path = f"../data/chunks/{prefix}_{target_language}.json"
translated_questions_file_path = (
    f"../data/question_answer_pairs/{prefix}_qa_pairs_{target_language}.json"
)

with open(translated_text_chunk_file_path, "w") as f:
    json.dump(text_chunks, f, indent=2)

with open(translated_questions_file_path, "w") as f:
    json.dump(questions, f, indent=2)